In [8]:
import datetime

import pandas as pd

from stock_common.lib import util
from stock_common.lib.database import Database
from stock_common.conf.config import Config
from stock_common.lib.storage import minio_client
from stock_recommender.data.sp500 import SP500

CONFIGS = Config.get_configs()
db = Database(CONFIGS)

In [9]:
%matplotlib inline

In [10]:
symbols = SP500().get_symbols_from_mongo()

if 'SPY' not in symbols:
    symbols.append('SPY')

In [26]:
if datetime.datetime.today().hour >= 16:
    end_date = datetime.date.today()
else:
    end_date = datetime.date.today() - datetime.timedelta(days=1)
#start_date = end_date - datetime.timedelta(days=50)
#print(start_date)
#print(end_date)

In [12]:
# pf = util.read_data(symbols, start_date, end_date)

In [13]:
pf_price = pd.read_hdf('../tmp/' + CONFIGS.PRICES_H5)

In [14]:
df_close = pf_price.Close.tail(21).dropna(axis=1)

In [28]:
end_date

datetime.date(2017, 10, 26)

In [35]:
end_date == df_close.index[-1].date()

True

In [ ]:
df_ma = df_close.rolling(window=20,center=False).mean().tail(2)
df_std = df_close.rolling(window=20,center=False).std().tail(2)
df_bollingerbands = (df_close.tail(2) - df_ma) / df_std

In [ ]:
df_close.tail(2)

In [ ]:
df_ma

In [ ]:
df_std

In [ ]:
df_bollingerbands

In [ ]:
todays_symbols = df_bollingerbands.columns.tolist()

In [ ]:
df_bollingerbands.sort_index(inplace=True)

In [ ]:
#     Bollinger value for the equity today <= -2.0
#     Bollinger value for the equity yesterday >= -2.0
#     Bollinger value for SPY today >= 1.2

In [ ]:
def date_to_int(date):
    return int(datetime.datetime.strftime(date, '%Y%m%d'))

In [ ]:
recommended_symbols = []
SPY = df_bollingerbands.iloc[1]['SPY']
date_to_sell = end_date + datetime.timedelta(days=5)
for symbol in todays_symbols:
    yesterday = df_bollingerbands.iloc[0][symbol]
    today = df_bollingerbands.iloc[1][symbol]
    if yesterday >= -2.0 and today <= -2.0 and SPY >= 1.2:
        recommended_symbols.append(symbol)

model_result = {
    'symbols': recommended_symbols,
    'date_to_buy': date_to_int(end_date),
    'holding_period': 5,
    'date_to_sell': date_to_int(date_to_sell),
    'model': 'bollinger',
    'created_at': date_to_int(end_date),
}

In [ ]:
SPY

In [ ]:
model_result

In [ ]:
df_close[recommended_symbols].plot.line()